Library

In [6]:
# Core Libraries
import os
import io
import base64
import warnings
from datetime import datetime

# Data Processing & Math
import numpy as np
from sklearn.model_selection import train_test_split

# Audio Processing
import librosa
import librosa.display

# Deep Learning
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

# Visualization
import matplotlib.pyplot as plt

# Web Framework
from flask import Flask, render_template_string, render_template, request
from werkzeug.utils import secure_filename


from IPython.display import Image, display
from pyngrok import ngrok
from dotenv import load_dotenv

load_dotenv('token.env')  # Load dari file .env
ngrok.set_auth_token(os.getenv("NGROK_AUTHTOKEN"))

Training model

In [ ]:
# Fungsi ekstrak fitur MFCC dengan padding yang konsisten
def extract_features(file_path, max_pad_len=500):  # Diperbesar menjadi 500
    try:
        audio, sample_rate = librosa.load(file_path, sr=22050, res_type='kaiser_fast')  # SR diset 22050 untuk konsistensi
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)

        # Padding atau truncate ke max_pad_len
        if mfccs.shape[1] > max_pad_len:
            mfccs = mfccs[:, :max_pad_len]
        else:
            pad_width = max_pad_len - mfccs.shape[1]
            mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')

        return mfccs
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

# Muat dataset dengan penanganan error yang lebih baik
def load_dataset(data_dir):
    features = []
    labels = []
    label_dict = {}

    # Cek semua kelas suara
    class_names = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]

    for label_idx, class_name in enumerate(class_names):
        class_dir = os.path.join(data_dir, class_name)
        label_dict[label_idx] = class_name

        for file_name in os.listdir(class_dir):
            if not file_name.lower().endswith(('.wav', '.mp3')):
                continue

            file_path = os.path.join(class_dir, file_name)
            mfccs = extract_features(file_path)

            if mfccs is not None:
                features.append(mfccs)
                labels.append(label_idx)

    # Konversi ke numpy array dengan pengecekan
    if not features:
        raise ValueError("Tidak ada file audio yang berhasil diproses!")

    features = np.array(features)
    labels = np.array(labels)

    return features, labels, label_dict

# Bangun model CNN
def build_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
    return model

def main():
    # Path dataset
    dataset_dir = "dataset_suara"

    try:
        # Muat dataset
        features, labels, label_dict = load_dataset(dataset_dir)
        print(f"Berhasil memuat {len(features)} sampel audio")

        # Reshape untuk CNN
        X = features[..., np.newaxis]  # Tambah dimensi channel
        X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

        # Latih model
        model = build_model(X_train.shape[1:], len(label_dict))
        history = model.fit(X_train, y_train,
                          epochs=50,
                          batch_size=32,
                          validation_data=(X_test, y_test))

        # Simpan model
        model_path = "dataset_suara/sound_model.h5"
        model.save(model_path)
        np.save("dataset_suara/label_dict.npy", label_dict)
        print(f"✅ Model disimpan di {model_path}")

    except Exception as e:
        print(f"Error utama: {str(e)}")

if __name__ == "__main__":
    main()

predict sound

In [2]:
import os
import io
import base64
import warnings
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
import tensorflow as tf

from flask import Flask, request, render_template
from werkzeug.utils import secure_filename

from pyngrok import ngrok
from dotenv import load_dotenv

load_dotenv('token.env')  # Load dari file .env
ngrok.set_auth_token(os.getenv("NGROK_AUTHTOKEN"))


warnings.filterwarnings('ignore')

app = Flask(__name__)

# Path konfigurasi
model_path = "dataset_suara/sound_model.h5"
label_dict_path = "dataset_suara/label_dict.npy"
image_dir = "GambarHasil"

UPLOAD_FOLDER = 'uploads'
ALLOWED_EXTENSIONS = {'wav', 'mp3', 'ogg'}
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024  # 16MB

os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)

# Load model & label
try:
    model = tf.keras.models.load_model(model_path)
    label_dict = np.load(label_dict_path, allow_pickle=True).item()
    print("✅ Model berhasil dimuat!")
    HEWAN_DIKENALI = list(label_dict.values())

    reference_images = {
        'anjing': os.path.join(image_dir, 'anjing.jpg'),
        'ayam': os.path.join(image_dir, 'ayam.jpg'),
        'burung': os.path.join(image_dir, 'burung.jpg'),
        'domba': os.path.join(image_dir, 'domba.jpg'),
        'keledai': os.path.join(image_dir, 'keledai.jpg'),
        'kodok': os.path.join(image_dir, 'kodok.jpg'),
        'kucing': os.path.join(image_dir, 'kucing.jpg'),
        'monkey': os.path.join(image_dir, 'monkey.jpg'),
        'sapi': os.path.join(image_dir, 'sapi.jpg'),
        'singa': os.path.join(image_dir, 'singa.jpg')
    }

except Exception as e:
    print(f"❌ Gagal memuat model: {e}")
    raise

# Cek ekstensi file
def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

# Ekstraksi fitur
def ekstrak_fitur(file_path, max_pad_len=500, n_mels=128, n_mfcc=40):
    try:
        audio, sr = librosa.load(file_path, sr=22050)
        audio = librosa.util.normalize(audio)
        mel = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=n_mels)
        mfccs = librosa.feature.mfcc(S=librosa.power_to_db(mel), n_mfcc=n_mfcc)

        if mfccs.shape[1] > max_pad_len:
            mfccs = mfccs[:, :max_pad_len]
        else:
            pad_width = max_pad_len - mfccs.shape[1]
            mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')

        return mfccs, audio, sr
    except Exception as e:
        print(f"Gagal memproses {file_path}: {e}")
        return None, None, None

# Visualisasi hasil prediksi
def buat_visualisasi(audio, sr, mfccs, prediksi, kelas_prediksi):
    plt.figure(figsize=(15, 8))
    plt.subplot(2, 2, 1)
    librosa.display.waveshow(audio, sr=sr, color='blue')
    plt.title('Gelombang Suara')
    plt.xlabel('Waktu (detik)')
    plt.ylabel('Amplitudo')

    plt.subplot(2, 2, 2)
    librosa.display.specshow(mfccs, x_axis='time', cmap='coolwarm')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Koefisien MFCC')

    plt.subplot(2, 1, 2)
    probabilitas = prediksi[0]
    warna = ['green' if h == kelas_prediksi else 'gray' for h in HEWAN_DIKENALI]
    bars = plt.barh(HEWAN_DIKENALI, probabilitas, color=warna)
    for bar in bars:
        width = bar.get_width()
        plt.text(width + 0.01, bar.get_y() + bar.get_height() / 2,
                 f'{width:.2%}', ha='left', va='center')

    plt.title('Probabilitas Prediksi')
    plt.xlim(0, 1)
    plt.tight_layout()

    buf = io.BytesIO()
    plt.savefig(buf, format='png', bbox_inches='tight')
    plt.close()
    buf.seek(0)

    return base64.b64encode(buf.getvalue()).decode('utf8')

# Prediksi suara
def prediksi_suara(file_path):
    try:
        mfccs, audio, sr = ekstrak_fitur(file_path)
        if mfccs is None:
            return None, "Gagal memproses file audio"

        mfccs_input = mfccs[np.newaxis, ..., np.newaxis]
        prediksi = model.predict(mfccs_input)
        kelas_prediksi = label_dict[np.argmax(prediksi)]

        plot_url = buat_visualisasi(audio, sr, mfccs, prediksi, kelas_prediksi)

        detail_prob = {
            label_dict[i]: f"{prob*100:.2f}%"
            for i, prob in enumerate(prediksi[0])
        }

        gambar_ref = None
        img_path = reference_images.get(kelas_prediksi.lower(), None)
        if img_path and os.path.exists(img_path):
            with open(img_path, "rb") as img_file:
                gambar_ref = "data:image/jpeg;base64," + base64.b64encode(img_file.read()).decode('utf-8')

        return {
            'hasil': kelas_prediksi,
            'plot': plot_url,
            'probabilitas': detail_prob,
            'waktu': datetime.now().strftime("%d/%m/%Y %H:%M:%S"),
            'gambar_referensi': gambar_ref
        }, None
    except Exception as e:
        return None, str(e)

# ROUTE
@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        if 'file' not in request.files or request.files['file'].filename == '':
            return render_template("hasil.html", error="Tidak ada file yang dipilih", hewan_dikenali=HEWAN_DIKENALI)

        file = request.files['file']
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(filepath)

            hasil, error = prediksi_suara(filepath)
            if error:
                return render_template("hasil.html", error=error, hewan_dikenali=HEWAN_DIKENALI)

            return render_template("hasil.html", hasil=hasil, hewan_dikenali=HEWAN_DIKENALI)
        else:
            return render_template("hasil.html", error="Format file tidak didukung.", hewan_dikenali=HEWAN_DIKENALI)

    return render_template("hasil.html", hewan_dikenali=HEWAN_DIKENALI)

# Jalankan Flask
if __name__ == '__main__':
    public_url = ngrok.connect(5000).public_url
    print(" * Aplikasi berjalan di:", public_url)
    app.run()


✅ Model berhasil dimuat!
 * Aplikasi berjalan di: https://ad78-2400-9800-584-e796-5847-2013-f412-59d1.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [24/May/2025 14:29:48] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/May/2025 14:29:50] "GET /favicon.ico HTTP/1.1" 404 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step


INFO:werkzeug:127.0.0.1 - - [24/May/2025 14:29:58] "POST / HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step


...

INFO:werkzeug:127.0.0.1 - - [24/May/2025 14:46:30] "POST / HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step


INFO:werkzeug:127.0.0.1 - - [24/May/2025 14:47:27] "POST / HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step


INFO:werkzeug:127.0.0.1 - - [24/May/2025 14:47:44] "POST / HTTP/1.1" 200 -
